In [1]:
import numpy as np

import pickle as pickle
import pandas as pd
import pyeeg as pe


In [8]:
# channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
channel=[36]
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 70 #Sampling rate of 128 Hz
subjectList = ['01']
#List of subjects

In [3]:
from sklearn.preprocessing import normalize

In [11]:
def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open('s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,39):
            # loop over 0-39 trails
            data = subject["data"][i]
            if i==0:
                data[36]=(data[36]-min(data[36]))/(max(data[36])-min(data[36]))
                print(data[36])

            # print(data.shape)
            # print(type(data))
            labels = subject["labels"][i]
            print(type(labels))
            print(labels.shape)
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
                
        meta = np.array(meta)
        print(meta.shape)
        np.save('s' + sub, meta, allow_pickle=True, fix_imports=True)

In [12]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

[0.24751048 0.24950213 0.24950213 ... 0.18509206 0.18509206 0.18509206]
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)
<class 'numpy.ndarray'>
(4,)


In [6]:
meta

NameError: name 'meta' is not defined

In [9]:
data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

for subjects in subjectList:
  

    with open('s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        for i in range (0,sub.shape[0]):
            if i % 8 == 0 or i % 8== 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
           
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

np.save('out/data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('out/label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)


np.save('out/data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('out/label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)
"""
np.save('out\data_validation', np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save('out\label_validation', np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)
"""

training dataset: (16653, 70) (16653, 4)
testing dataset: (2379, 70) (2379, 4)


'\nnp.save(\'out\\data_validation\', np.array(data_validation), allow_pickle=True, fix_imports=True)\nnp.save(\'out\\label_validation\', np.array(label_validation), allow_pickle=True, fix_imports=True)\nprint("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)\n'

In [10]:
data= []
label = []
for subjects in subjectList:
  

    with open('s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        for i in range (0,sub.shape[0]):
          data.append(sub[i][0])
          label.append(sub[i][1])
np.save('data', np.array(data), allow_pickle=True, fix_imports=True)
np.save('label', np.array(label), allow_pickle=True, fix_imports=True)

In [11]:
df=pd.DataFrame(data=data)
df.to_csv("data.csv",index=False)

df1=pd.DataFrame(data=label)
df1.to_csv("label.csv",index=False)

In [12]:

data1=pd.read_csv("data.csv")



In [17]:
data1

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,1467.141307,957.354712,975.500368,886.561347,257.611040,1729.379210,1079.293596,1017.848244,950.011987,277.194922,...,1616.579471,1015.132346,769.122057,991.446909,286.112522,1897.870710,1178.996354,740.014784,722.807957,218.773480
1,1402.080988,872.471171,1069.522141,1008.312775,741.738698,1642.212773,959.802426,1016.890625,1016.753121,486.161277,...,1639.170401,924.285483,828.084355,971.772255,189.171448,1902.103188,1122.875330,761.432035,768.601395,359.695638
2,1569.859186,895.845443,1035.124465,918.612673,113.293654,1678.108287,964.516236,989.247998,968.499790,138.022717,...,1546.708333,986.628413,876.599937,997.347209,244.353918,2057.569881,1161.216557,736.055635,768.684543,240.153545
3,1497.247865,851.437981,959.506731,882.725062,260.333692,1537.752623,911.636878,936.074910,956.197358,383.036705,...,1452.084801,914.005947,814.784673,1033.127883,229.448013,1989.591808,1069.121464,686.572034,697.596546,50.708541
4,1314.476209,919.669964,983.308354,873.276553,352.814075,1351.133996,1007.996757,1000.436238,982.752578,241.910344,...,1433.935229,901.104793,794.645651,1000.369776,199.378771,1739.514883,1011.118559,824.067509,737.803041,496.043021
5,1309.176522,1099.660539,907.724288,906.404042,136.097922,1385.182969,1129.747708,882.812046,982.846560,276.781474,...,1365.900057,850.916617,793.859526,964.361406,109.867268,1767.650766,1049.157100,721.676108,728.220122,69.100969
6,1237.431243,1063.702677,891.905701,908.705425,161.983532,1365.409676,1099.636139,914.707424,1004.232879,172.275619,...,1513.914066,886.989082,790.535683,1020.391807,217.897455,1854.166128,1027.467328,726.360878,731.771853,158.032036
7,1244.898723,1065.962243,906.101782,857.912422,244.263268,1360.966105,1115.665481,944.364747,920.383638,238.874215,...,1774.242061,904.575585,802.758812,1011.233901,441.401355,1807.070805,1044.502540,745.331881,781.732415,204.585627
8,1184.047516,1074.186615,901.935113,881.755745,32.730709,1330.153861,1154.447798,949.085612,950.114493,223.932970,...,1898.861208,887.958978,812.874698,1031.445608,299.160050,1850.436666,1063.655101,768.905344,797.318123,222.875457
9,1531.760433,1078.439074,954.109606,848.902407,472.438583,1552.677502,1187.985593,966.881136,957.833509,533.805546,...,1858.507445,846.373445,875.794127,1054.069751,300.625325,1999.667188,1092.301389,739.447488,809.243690,490.400066


In [16]:
label1=pd.read_csv("label.csv")
label1

,0,1,2,3
0,7.71,7.60,6.90,7.83
1,7.71,7.60,6.90,7.83
2,7.71,7.60,6.90,7.83
3,7.71,7.60,6.90,7.83
4,7.71,7.60,6.90,7.83
5,7.71,7.60,6.90,7.83
6,7.71,7.60,6.90,7.83
7,7.71,7.60,6.90,7.83
8,7.71,7.60,6.90,7.83
9,7.71,7.60,6.90,7.83


In [18]:
x=data1.values

In [19]:
label1.isnull().sum()

0    0
1    0
2    0
3    0
dtype: int64

In [20]:
y_val=label1

In [21]:


y_val



,0,1,2,3
0,7.71,7.60,6.90,7.83
1,7.71,7.60,6.90,7.83
2,7.71,7.60,6.90,7.83
3,7.71,7.60,6.90,7.83
4,7.71,7.60,6.90,7.83
5,7.71,7.60,6.90,7.83
6,7.71,7.60,6.90,7.83
7,7.71,7.60,6.90,7.83
8,7.71,7.60,6.90,7.83
9,7.71,7.60,6.90,7.83


In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)




In [25]:
y_val

,0,1,2,3
0,7.71,7.60,6.90,7.83
1,7.71,7.60,6.90,7.83
2,7.71,7.60,6.90,7.83
3,7.71,7.60,6.90,7.83
4,7.71,7.60,6.90,7.83
5,7.71,7.60,6.90,7.83
6,7.71,7.60,6.90,7.83
7,7.71,7.60,6.90,7.83
8,7.71,7.60,6.90,7.83
9,7.71,7.60,6.90,7.83


In [26]:
x.shape[0]

19032

In [27]:
x.shape[1]

70

In [28]:
x = np.reshape(x, (x.shape[0],1,x.shape[1]))

In [31]:
x.shape

(19032, 1, 70)

In [30]:
y_val.shape



(19032, 4)

In [32]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_val, test_size = 0.2, random_state = 4)

In [33]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(15225, 1, 70)
(15225, 4)
(3807, 1, 70)
(3807, 4)


In [34]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM,BatchNormalization,Activation
model = Sequential()
model.add(LSTM(512, batch_input_shape = (None, None, x.shape[2]),return_sequences=True))
#model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(LSTM(256,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(LSTM(128,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(LSTM(64,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(LSTM(32,activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Dense(4))

rmsprop =keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08)
model.compile(loss='mean_squared_error',
                  optimizer=rmsprop,
                  metrics=['accuracy'])
#adam = keras.optimizers.Adam(lr=0.5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 512)         1193984   
_________________________________________________________________
batch_normalization (BatchNo (None, None, 512)         2048      
_________________________________________________________________
dropout (Dropout)            (None, None, 512)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         787456    
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 256)         1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 128)         1

In [ ]:
7